In [1]:
!pip install fake_useragent

In [2]:
import re
import csv
import time
import json
import random
import requests
import numpy as np
import pandas as pd
from tqdm import tqdm
import concurrent.futures
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
from concurrent.futures import ThreadPoolExecutor, as_completed

In [3]:
# Подключение к google drive

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


### Рейтинг банков - https://www.sravni.ru/banki/rating/klientskij/?userRatingType=all

Результаты будем сохранять в файл **sravni_banks.csv**.

* place - место в рейтинге
* name - название банка (на русском языке)
* eng_name - название банка (на английском языке)
* rating - рейтинг
* responses - кол-во отзывов
* answers - кол-во ответов банка
* problems - кол-во решенных проблем
* bank_id - id банка

In [4]:
%%time

df = pd.DataFrame({'place':[], 'name':[], 'eng_name':[], 'rating':[], 'responses':[], 'answers':[], 'problems':[], 'bank_id':[]})

url = 'https://www.sravni.ru/banki/rating/klientskij/?userRatingType=all'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

items = soup.find_all('tr', {'role':'row'})

i = 1
for item in items[1:]:

  try:
    place = i

    name = item.find('td', {'class':'cell_cell--name__bpHYG'}).text
    eng_name = re.fullmatch(r'/bank/(.+?)/otzyvy/', item.find('a')['href']).group(1)
    rating = item.find('td', {'class':'cell_cell--rating__KyQ3_'}).text
    responses = item.find('td', {'class':'cell_cell--reviewsCount__4HQLJ'}).text
    answers = item.find('td', {'class':'cell_cell--responseCount__PcrpN'}).text
    problems = item.find('td', {'class':'cell_cell--solvedProblems__LUIt5'}).text
    bank_id = re.findall(r"square/(.*?)\.svg", item.find('img')['src'])[0]

    df.loc[len(df)] = [i, name, eng_name, rating, responses, answers, problems, bank_id]

  except:
    pass

  i += 1

df.to_csv('/content/drive/MyDrive/bank_reviews_nlp/sravni_banks.csv', index=False)

df.head()

CPU times: user 593 ms, sys: 13.9 ms, total: 607 ms
Wall time: 2.05 s


,place,name,eng_name,rating,responses,answers,problems,bank_id
0,1,Альфа-Банк,alfa-bank,4.91,69449,43700,2245,5bb4f767245bc22a520a60fb
1,2,Совкомбанк,sovkombank,4.89,42514,40920,995,5bb4f767245bc22a520a5fd5
2,3,Т-Банк,tinkoff-bank,4.65,55916,55858,1319,5bb4f769245bc22a520a6353
3,4,Банк Точка,tochka,4.47,2837,2816,26,5bb4f76b245bc22a520a6723
4,5,Центр-инвест,centr-invest,4.21,1862,1476,3,5bb4f76a245bc22a520a64db


### Отзывы о банках - https://www.sravni.ru/banki/otzyvy/?filterBy=all&orderBy=byDate

Результаты будем сохранять в файл - sravni_ru_reviews.csv.

 - url - ссылка на отзыв
 - review_date - дата и время оставления отзыва
 - user_name - имя пользователя
 - user_city - город пользователя
 - user_city_full - расширенный город пользователя (область/город и тд)
 - review_title - тема обращения
 - review_text - текст обращения
 - review_status - статус обращения (проверяется/проверен)
 - problem_status - статус проблемы (решена/не решена)
 - rating - оценка
 - review_tag - тег обращения (кредитные карты/дебетовые карты и тд)
 - bank_name - назание банка (на русском языке)
 - is_bank_ans - наличие ответа банка (True/False)

In [8]:
# Словарь для поиска названия банка по его id

banki = pd.read_csv('/content/drive/MyDrive/bank_reviews_nlp/sravni_banks.csv')
banki_dct = banki[['name', 'eng_name', 'bank_id']].set_index('bank_id').to_dict('index')

In [9]:
%%time

ua = UserAgent()
user_agents = [ua.chrome, ua.google, ua['google chrome'], ua.firefox, ua.ff, ua.safari]

date_start = '2023-01-01T00:00:00+00:00'
date_finish = '2023-08-26T00:00:00+00:00'
review_date = '2024-06-28T00:00:00+00:00'   # нужна для первого запуска цикла while


with open('/content/drive/MyDrive/bank_reviews_nlp/sravni_ru_reviews.csv', "w", encoding='utf-8') as w_file:
  file_writer = csv.writer(w_file, delimiter = ",", lineterminator="\n")
  file_writer.writerow(['url', 'review_date', 'user_name', 'user_city', 'user_city_full', 'review_title', 'review_text',
                        'review_status', 'problem_status', 'rating', 'review_tag', 'bank_name', 'is_bank_ans'])

  page = 7140

  while review_date >= date_start:

    url = f'https://www.sravni.ru/proxy-reviews/reviews?filterBy=all&fingerPrint=1d345dd221ef718448c6bef7fc795d47&isClient=false&locationRoute=&newIds=true&orderBy=byDate&pageIndex={page}&pageSize=10&reviewObjectId&reviewObjectType=banks&specificProductId=&tag=&withVotes=true'
    headers = {'User-Agent': random.choice(user_agents)}
    response = requests.get(url, headers=headers, timeout=30)
    data = json.loads(response.text)

    for data in data['items']:

      review_date = data['createdToMoscow']

      if review_date <= date_finish:

        try:
          response_id = data['id']
          bank_id =  data['reviewObjectId']
          bank_name_eng = banki_dct[f'{bank_id}']['eng_name']
          url = f'https://www.sravni.ru/bank/{bank_name_eng}/otzyvy/{response_id}/'

          user_name = data['authorName']
          user_city = data['locationData']['name']
          user_city_full = data['locationData']['fullName']
          review_title = data['title']
          review_text = data['text']
          review_status = data['ratingStatus']
          problem_status = data['problemSolved']
          rating = data['rating']
          review_tag = data['reviewTag']
          bank_name = banki_dct[f'{bank_id}']['name']
          is_bank_ans = data['hasCompanyResponse']

          file_writer.writerow([url, review_date, user_name, user_city, user_city_full, review_title, review_text,
                                review_status, problem_status, rating, review_tag, bank_name, is_bank_ans])
        except:
          pass

    page += 1

На скачивание всего датасета понадобилось около 6 часов.